<a href="https://colab.research.google.com/github/KBVBrahmarao/datasciencecoursera/blob/main/Lab4_InLab1_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAB4:
In-Lab:
1. Implement a MapReduce program to filter records based on a given condition.
Example: Filter only the records where salary > 50,000, or lines containing a specific
keyword (e.g., "ERROR").
 Write the Mapper logic for filtering.
 Use an Identity Reducer or write your own reducer.
 Run the program and capture output screenshots.

In [1]:
#STEP 1 — Install Java
!apt-get update -qq
!apt-get install -y openjdk-17-jdk-headless

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-17-jdk-headless is already the newest version (17.0.17+10-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [2]:
#Check Java:
!java -version

openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)


In [3]:
#STEP 2 — Set Java Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"] += ":/usr/lib/jvm/java-17-openjdk-amd64/bin"

In [4]:
#STEP 3 — Download and Extract Hadoop
!wget -q https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz
!mv hadoop-3.3.6 /content/hadoop

In [5]:
#STEP 4 — Set Hadoop Environment Variables
os.environ["HADOOP_HOME"] = "/content/hadoop"
os.environ["HADOOP_CONF_DIR"] = "/content/hadoop/etc/hadoop"
os.environ["PATH"] += ":/content/hadoop/bin:/content/hadoop/sbin"

In [7]:
#Extra STEP 1 — Set JAVA_HOME again
!export HADOOP_HOME=/content/Hadoop
!export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
!export PATH=$PATH:$JAVA_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

In [9]:
#Extra Step 2 — Verify directories
!ls /content/Hadoop/bin

ls: cannot access '/content/Hadoop/bin': No such file or directory


In [10]:
#ExtraSTEP 3 — Check Hadoop version again
!export HADOOP_HOME=/content/Hadoop && export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64 && export PATH=$PATH:$JAVA_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin && hadoop version

ERROR: Cannot execute /content/Hadoop/libexec/hadoop-config.sh.


In [11]:
#STEP 5 — Correct Hadoop Configuration Files - core-site.xml
%%writefile /content/hadoop/etc/hadoop/core-site.xml
<?xml version="1.0"?>
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>file:///</value>
    </property>
</configuration>

Overwriting /content/hadoop/etc/hadoop/core-site.xml


In [12]:
#Step 6 - Create hdfs-site.xml (even though HDFS cannot run in Colab)
#Step 6 - Create hdfs-site.xml (even though HDFS cannot run in Colab)
%%writefile /content/hadoop/etc/hadoop/hdfs-site.xml
<?xml version="1.0"?>
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>

Overwriting /content/hadoop/etc/hadoop/hdfs-site.xml


In [13]:
#STEP  7A — Verify that files are correct
!cat /content/hadoop/etc/hadoop/core-site.xml
!cat /content/hadoop/etc/hadoop/hdfs-site.xml

<?xml version="1.0"?>
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>file:///</value>
    </property>
</configuration>
<?xml version="1.0"?>
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>


In [14]:
#Step 7B - Verify the files were created
!ls /content/hadoop/etc/hadoop

capacity-scheduler.xml		  kms-log4j.properties
configuration.xsl		  kms-site.xml
container-executor.cfg		  log4j.properties
core-site.xml			  mapred-env.cmd
hadoop-env.cmd			  mapred-env.sh
hadoop-env.sh			  mapred-queues.xml.template
hadoop-metrics2.properties	  mapred-site.xml
hadoop-policy.xml		  shellprofile.d
hadoop-user-functions.sh.example  ssl-client.xml.example
hdfs-rbf-site.xml		  ssl-server.xml.example
hdfs-site.xml			  user_ec_policies.xml.template
httpfs-env.sh			  workers
httpfs-log4j.properties		  yarn-env.cmd
httpfs-site.xml			  yarn-env.sh
kms-acls.xml			  yarnservice-log4j.properties
kms-env.sh			  yarn-site.xml


In [15]:
#STEP 7C — Set correct config directory
%%bash
export HADOOP_HOME=/content/hadoop
export HADOOP_CONF_DIR=/content/hadoop/etc/hadoop
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export PATH=$PATH:$JAVA_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

hadoop version

Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /content/hadoop/share/hadoop/common/hadoop-common-3.3.6.jar


In [20]:
#STEP 7D — Delete old output
!rm -rf filteroutput

In [17]:
#NEXT IMPORTANT STEPs
#STEP 7 - Fix log4j warning (optional but recommended)
%%bash
cat > /content/hadoop/etc/hadoop/log4j.properties <<'EOF'
log4j.rootLogger=INFO, console
log4j.appender.console=org.apache.log4j.ConsoleAppender
log4j.appender.console.target=System.err
log4j.appender.console.layout=org.apache.log4j.PatternLayout
log4j.appender.console.layout.ConversionPattern=%d{ISO8601} %-5p %c: %m%n
EOF

In [18]:
#STEP 8 — Run a Sample MapReduce Job (IMPORTANT TEST)
#Create an input file:
%%writefile employees.txt
John,45000
Mary,72000
David,55000
Kiran,38000
Priya,90000

Writing employees.txt


In [24]:
# Mapper in Python (filters salary > 50000)
%%writefile mapper_filter.py
#!/usr/bin/env python3
import sys

THRESHOLD = 50000  # filtering condition

for line in sys.stdin:
    line = line.strip()
    if not line:
        continue

    name, salary = line.split(",")
    salary = int(salary)

    if salary > THRESHOLD:
        # Emit key-value pair
        print(f"{name}\t{salary}")

Writing mapper_filter.py


In [26]:
# Reducer in Python (pass-through)
%%writefile reducer_filter.py
#!/usr/bin/env python3
import sys

for line in sys.stdin:
    print(line.strip())

Overwriting reducer_filter.py


In [27]:
#STEP 8.1
%%bash
hdfs dfs -mkdir -p /filterinput
hdfs dfs -put -f employees.txt /filterinput
hdfs dfs -ls /filterinput

Found 1 items
-rw-r--r--   1 root root         58 2025-12-31 09:27 /filterinput/employees.txt


In [28]:
#Run MapReduce Example (Line Length)
#Hadoop already provides example JARs.
%%bash
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export HADOOP_HOME=/content/hadoop
export PATH=$PATH:$JAVA_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

hdfs dfs -rm -r /filteroutput

hadoop jar \
$HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.6.jar \
grep /filterinput /filteroutput '.*'

Deleted /filteroutput


2025-12-31 09:27:18,595 INFO  org.apache.hadoop.conf.Configuration.deprecation: io.bytes.per.checksum is deprecated. Instead, use dfs.bytes-per-checksum
2025-12-31 09:27:19,932 INFO  org.apache.hadoop.metrics2.impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-12-31 09:27:20,102 INFO  org.apache.hadoop.metrics2.impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-12-31 09:27:20,102 INFO  org.apache.hadoop.metrics2.impl.MetricsSystemImpl: JobTracker metrics system started
2025-12-31 09:27:20,358 INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat: Total input files to process : 1
2025-12-31 09:27:20,391 INFO  org.apache.hadoop.mapreduce.JobSubmitter: number of splits:1
2025-12-31 09:27:20,646 INFO  org.apache.hadoop.mapreduce.JobSubmitter: Submitting tokens for job: job_local703189619_0001
2025-12-31 09:27:20,647 INFO  org.apache.hadoop.mapreduce.JobSubmitter: Executing with tokens: []
2025-12-31 09:27:20,941 INFO  org.apache.had

In [29]:
#View Output
%%bash
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export HADOOP_HOME=/content/hadoop
export PATH=$PATH:$JAVA_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

hdfs dfs -cat /filteroutput/part-r-00000

5	
1	Priya,90000
1	Mary,72000
1	Kiran,38000
1	John,45000
1	David,55000


In [30]:
!cat employees.txt | python3 mapper_filter.py | python3 reducer_filter.py

Mary	72000
David	55000
Priya	90000
